# BATS/TBATS Forecasting (Multiple Seasonality)

BATS (Box-Cox, ARMA errors, Trend, Seasonal) and TBATS (adds **Trigonometric** seasonality)
handle complex, multiple seasonal patterns that appear in hourly, daily, or weekly data.


## Model ingredients
After a Box-Cox transform, the model is often written as:
\[y_t^{(\lambda)} = \ell_{t-1} + \phi b_{t-1} + \sum_{i=1}^m s_{t,i} + d_t\]
where $s_{t,i}$ are seasonal components and $d_t$ follows an ARMA error process.
TBATS uses Fourier (sine/cosine) terms to represent each seasonal component, which is
efficient for long or multiple seasonalities.


In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

rng = np.random.default_rng(42)
n = 24 * 28
t = np.arange(n)
season_daily = 10 * np.sin(2 * np.pi * t / 24)
season_weekly = 5 * np.sin(2 * np.pi * t / (24 * 7))
trend = 0.02 * t
noise = rng.normal(0, 1.5, size=n)
y = 50 + trend + season_daily + season_weekly + noise

index = pd.date_range("2025-01-01", periods=n, freq="H")
series = pd.Series(y, index=index, name="y")

fig = px.line(series, title="Synthetic series with daily + weekly seasonality")
fig.update_layout(xaxis_title="Time", yaxis_title="Value")
fig.show()


## Trigonometric seasonality via Fourier terms
TBATS represents seasonality using Fourier bases. The first few sine/cosine terms often
capture most of the periodic structure while keeping the model compact.


In [ ]:
import plotly.graph_objects as go

period = 24
k = 3  # number of Fourier pairs
t_short = np.arange(period)

fourier_terms = []
for i in range(1, k + 1):
    fourier_terms.append(np.sin(2 * np.pi * i * t_short / period))
    fourier_terms.append(np.cos(2 * np.pi * i * t_short / period))

fig = go.Figure()
for idx, term in enumerate(fourier_terms, 1):
    fig.add_trace(go.Scatter(x=t_short, y=term, mode="lines", name=f"term {idx}"))
fig.update_layout(title="Fourier basis terms for daily seasonality", xaxis_title="Hour")
fig.show()


## sktime inventory for BATS/TBATS
BATS/TBATS wrappers may require optional dependencies. Use the registry to confirm
availability in your environment.


In [ ]:
try:
    import pandas as pd
    from sktime.registry import all_estimators

    ests = all_estimators(estimator_types="forecaster", as_dataframe=True)
    mask = (
        ests["name"].str.contains("BATS|TBATS", case=False, na=False)
        | ests["module"].str.contains("tbats|bats", case=False, na=False)
    )
    print(ests.loc[mask, ["name", "module"]].sort_values("name").to_string(index=False))
except Exception as exc:
    print("sktime is not installed or registry lookup failed:", exc)


## When to use
- Multiple or long seasonalities (hourly + weekly, daily + yearly, etc.).
- Strong periodic structure with non-integer periods (handled via Fourier terms).
- Need a classical, interpretable alternative to deep learning baselines.
